# Incremental Proper Orthogonal Decomposition based reduced basis generation applied to Navier-Stokes equations

In this notebook, we demonstrate the incremental proper orthogonal decomposition (iPOD) for reduced basis generation on the example of the Navier-Stokes equations. Its implementation is based on and depicted in our [MORe DWR paper](https://doi.org/10.48550/arXiv.2304.01140). In summary, the iPOD can be interpreted as a trimmed version of the incremental trunctated Singular Value Decomposition introduced in [[Brand (2006)](https://www.sciencedirect.com/science/article/pii/S0024379505003812), [Brand (2002)](https://link.springer.com/chapter/10.1007/3-540-47969-4_47)]. Its use case to fluid flows and its highly parallizability is highlighted in [[Kühl et al.](https://arxiv.org/abs/2302.09149)] and demonstrates its attractivity for on-the-fly reduced basis generation and compression of problems of high dimensionality where storing the snapshot matrix is expensive or impossible. Thus, the method can also be considered as an alternative to checkpointing techniques.

In [ ]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

## Routine for incremental Proper Orthogonal Decomposition (iPOD)

In [ ]:
import numpy as np
import scipy

def iPOD(
    POD,  # POD basis
    snapshot,  # new snapshot to be added to POD basis
    bunch_matrix,  # bunch matrix
    bunch_size,  # desired size of bunch matrix
    singular_values,  # singular values of POD basis
    total_energy,  # total energy of POD basis
    energy_content,  # desired energy content
):

    if bunch_matrix.shape[1] == 0:
        # initialize bunch matrix if empty
        bunch_matrix = snapshot.reshape(-1, 1)  # np.empty([np.shape(snapshot)[0], 0])
    else:
        # concatenate new snapshot to bunch matrix
        bunch_matrix = np.hstack((bunch_matrix, snapshot.reshape(-1, 1)))

    # add energy of new snapshot to total energy
    total_energy += np.dot((snapshot), (snapshot))

    # check bunch_matrix size to decide if to update POD
    if bunch_matrix.shape[1] == bunch_size:
        # initialize POD with first bunch matrix
        if POD.shape[1] == 0:
            POD, S, _ = scipy.linalg.svd(bunch_matrix, full_matrices=False)

            # compute the number of POD modes to be kept
            r = 0
            while (np.dot(S[0:r], S[0:r]) / total_energy <= energy_content) and (
                r <= np.shape(S)[0]
            ):
                r += 1

            singular_values = S[0:r]
            POD = POD[:, 0:r]
        # update POD with  bunch matrix
        else:
            M = np.dot(POD.T, bunch_matrix)
            P = bunch_matrix - np.dot(POD, M)

            Q_p, R_p = scipy.linalg.qr(P, mode="economic")
            Q_q = np.hstack((POD, Q_p))

            S0 = np.vstack(
                (
                    np.diag(singular_values),
                    np.zeros((np.shape(R_p)[0], np.shape(singular_values)[0])),
                )
            )
            MR_p = np.vstack((M, R_p))
            K = np.hstack((S0, MR_p))

            # check the orthogonality of Q_q heuristically
            if np.inner(Q_q[:, 0], Q_q[:, -1]) >= 1e-10:
                Q_q, R_q = scipy.linalg.qr(Q_q, mode="economic")
                K = np.matmul(R_q, K)

            # inner SVD of K
            U_k, S_k, _ = scipy.linalg.svd(K, full_matrices=False)

            # compute the number of POD modes to be kept
            r = 0 
            while (np.dot(S_k[0:r], S_k[0:r]) / total_energy <= energy_content) and (
                r < np.shape(S_k)[0]
            ):
                r += 1

            singular_values = S_k[0:r]
            POD = np.matmul(Q_q, U_k[:, 0:r])

        # empty bunch matrix after update
        bunch_matrix = np.empty([np.shape(bunch_matrix)[0], 0])

    return POD, bunch_matrix, singular_values, total_energy


## Solving the Full Order Model with interface to iPOD basis enrichment

We borrowed the full-order FEM implementation from the FEniCS Project's [incompressible NSE tutorial](https://fenicsproject.org/pub/tutorial/html/._ftut1009.html). 

With regard to the integration of the iPOD method, since the iPOD only needs the newly obtained snapshots to enrich the reduced basis it can be easily integrated in the full-order solving process without interfering or impacting the FE solves. Note that reduced bases for velocity and pressure are generated separately.

After the FE simulation we further compare the impact of the iPOD to the overall computational time and plot the POD basis size for velocity and pressure, respectively.

In [ ]:
from fenics import *
from mshr import *
import matplotlib.pyplot as plt
import time

T = 5.0            # final time
num_steps = 5000 # number of time steps
dt = T / num_steps # time step size
mu = 0.001         # dynamic viscosity
rho = 1            # density

# Create mesh
channel = Rectangle(Point(0, 0), Point(2.2, 0.41))
cylinder = Circle(Point(0.2, 0.2), 0.05)
domain = channel - cylinder
mesh = generate_mesh(domain, 64)

# Define function spaces
V = VectorFunctionSpace(mesh, 'P', 2)
Q = FunctionSpace(mesh, 'P', 1)

# Define boundaries
inflow   = 'near(x[0], 0)'
outflow  = 'near(x[0], 2.2)'
walls    = 'near(x[1], 0) || near(x[1], 0.41)'
cylinder = 'on_boundary && x[0]>0.1 && x[0]<0.3 && x[1]>0.1 && x[1]<0.3'

# Define inflow profile
inflow_profile = ('4.0*1.5*x[1]*(0.41 - x[1]) / pow(0.41, 2)', '0')

# Define boundary conditions
bcu_inflow = DirichletBC(V, Expression(inflow_profile, degree=2), inflow)
bcu_walls = DirichletBC(V, Constant((0, 0)), walls)
bcu_cylinder = DirichletBC(V, Constant((0, 0)), cylinder)
bcp_outflow = DirichletBC(Q, Constant(0), outflow)
bcu = [bcu_inflow, bcu_walls, bcu_cylinder]
bcp = [bcp_outflow]

# Define trial and test functions
u = TrialFunction(V)
v = TestFunction(V)
p = TrialFunction(Q)
q = TestFunction(Q)

# Define functions for solutions at previous and current time steps
u_n = Function(V)
u_  = Function(V)
p_n = Function(Q)
p_  = Function(Q)

# Define expressions used in variational forms
U  = 0.5*(u_n + u)
n  = FacetNormal(mesh)
f  = Constant((0, 0))
k  = Constant(dt)
mu = Constant(mu)
rho = Constant(rho)

# Define symmetric gradient
def epsilon(u):
    return sym(nabla_grad(u))

# Define stress tensor
def sigma(u, p):
    return 2*mu*epsilon(u) - p*Identity(len(u))

# Define variational problem for step 1
F1 = rho*dot((u - u_n) / k, v)*dx \
   + rho*dot(dot(u_n, nabla_grad(u_n)), v)*dx \
   + inner(sigma(U, p_n), epsilon(v))*dx \
   + dot(p_n*n, v)*ds - dot(mu*nabla_grad(U)*n, v)*ds \
   - dot(f, v)*dx
a1 = lhs(F1)
L1 = rhs(F1)

# Define variational problem for step 2
a2 = dot(nabla_grad(p), nabla_grad(q))*dx
L2 = dot(nabla_grad(p_n), nabla_grad(q))*dx - (1/k)*div(u_)*q*dx

# Define variational problem for step 3
a3 = dot(u, v)*dx
L3 = dot(u_, v)*dx - k*dot(nabla_grad(p_ - p_n), v)*dx

# Assemble matrices
A1 = assemble(a1)
A2 = assemble(a2)
A3 = assemble(a3)

# Apply boundary conditions to matrices
[bc.apply(A1) for bc in bcu]
[bc.apply(A2) for bc in bcp]

# Initialize POD setting
POD = {"velocity": np.empty([0,0]), "pressure": np.empty([0,0])}
singular_values = {"velocity": np.empty([0]), "pressure": np.empty([0])}
total_energy = {"velocity": 0., "pressure": 0.}
bunch_matrix = {"velocity": np.empty([0,0]), "pressure": np.empty([0,0])}
BUNCH_SIZE = 20
ENERGY_CONTENT = 0.9999
snapshot_matrix = {"velocity": np.empty([0,0]), "pressure": np.empty([0,0])}

# performance timer
time_FEM = 0.0
time_iPOD = 0.0

# Time-stepping
t = 0
for n in range(num_steps):

    if n % 100 == 0:
      print(f"{n} from {num_steps} time steps")

    # Update current time
    t += dt
    time_start = time.time()
    # Step 1: Tentative velocity step
    b1 = assemble(L1)
    [bc.apply(b1) for bc in bcu]
    solve(A1, u_.vector(), b1, 'bicgstab', 'hypre_amg')

    # Step 2: Pressure correction step
    b2 = assemble(L2)
    [bc.apply(b2) for bc in bcp]
    solve(A2, p_.vector(), b2, 'bicgstab', 'hypre_amg')

    # Step 3: Velocity correction step
    b3 = assemble(L3)
    solve(A3, u_.vector(), b3, 'cg', 'sor')

    if n == 0:
      snapshot_matrix["velocity"] = u_.vector().get_local().reshape(-1, 1)
      snapshot_matrix["pressure"] = p_.vector().get_local().reshape(-1, 1)
    else:
      snapshot_matrix["velocity"] = np.hstack((snapshot_matrix["velocity"], u_.vector().get_local().reshape(-1, 1)))
      snapshot_matrix["pressure"] = np.hstack((snapshot_matrix["pressure"], p_.vector().get_local().reshape(-1, 1)))

    time_FEM += time.time() - time_start

    time_start = time.time()
    POD["velocity"], bunch_matrix["velocity"], singular_values["velocity"], \
          total_energy["velocity"] = iPOD(POD["velocity"], \
          u_.vector().get_local(), bunch_matrix["velocity"], BUNCH_SIZE, \
          singular_values["velocity"], total_energy["velocity"], ENERGY_CONTENT)

    POD["pressure"], bunch_matrix["pressure"], singular_values["pressure"], \
          total_energy["pressure"] = iPOD(POD["pressure"], \
          p_.vector().get_local(), bunch_matrix["pressure"], BUNCH_SIZE, \
          singular_values["pressure"], total_energy["pressure"], ENERGY_CONTENT)
    time_iPOD += time.time() - time_start 

    # Update previous solution
    u_n.assign(u_)
    p_n.assign(p_)

print(f"Time FEM:  {time_FEM}s")
print(f"Time iPOD: {time_iPOD}s")
print(f"Overhead due to iPOD: {(time_iPOD+time_FEM)/time_FEM*100-100}%")

print(f"Velocity POD size: {POD['velocity'].shape[1]}")
print(f"Pressure POD size: {POD['pressure'].shape[1]}")

## Reduced Snapshot Matrix

In [ ]:
# reduced snapshot matrix
snapshot_matrix_red = {"velocity": 0., "pressure": 0}
snapshot_matrix_red["velocity"]= \
  np.matmul(POD["velocity"],np.matmul(POD["velocity"].T,snapshot_matrix["velocity"]))
snapshot_matrix_red["pressure"]= \
  np.matmul(POD["pressure"],np.matmul(POD["pressure"].T,snapshot_matrix["pressure"]))

## Comparison of the full and reduced solutions

In [ ]:
# Plotting 
from matplotlib.animation import FuncAnimation
from matplotlib.ticker import MaxNLocator, FormatStrFormatter
from IPython.display import HTML, display
import os

if not os.path.exists("images"):
    os.makedirs("images")

u_fom = Function(V)
u_rom = Function(V)
u_diff = Function(V)

p_fom = Function(Q)
p_rom = Function(Q)
p_diff = Function(Q)

size_colorbar = 0.7
pad_space = 0.2
skip_frames = 50
for n in range(0,num_steps,skip_frames):
  u_fom.vector().set_local(snapshot_matrix["velocity"][:, n])
  u_rom.vector().set_local(snapshot_matrix_red["velocity"][:,n])
  u_diff.vector().set_local(snapshot_matrix["velocity"][:,n]-snapshot_matrix_red["velocity"][:,n])

  p_fom.vector().set_local(snapshot_matrix["pressure"][:, n])
  p_rom.vector().set_local(snapshot_matrix_red["pressure"][:,n])
  p_diff.vector().set_local(snapshot_matrix["pressure"][:,n]-snapshot_matrix_red["pressure"][:,n])

  fig, axes = plt.subplots(3, 2, figsize=(10, 7))  

  plt.subplot(3, 2, 1)
  c = plot(sqrt(dot(u_fom,u_fom)), title='Velocity - FOM')
  plt.colorbar(c, orientation='horizontal', ticks=MaxNLocator(nbins=3), format=FormatStrFormatter('%.2f'), pad=pad_space, shrink=size_colorbar)
  plt.subplot(3, 2, 3)
  c = plot(sqrt(dot(u_rom,u_rom)), title='Velocity - ROM')
  plt.colorbar(c, orientation='horizontal', ticks=MaxNLocator(nbins=3), format=FormatStrFormatter('%.2f'), pad=pad_space, shrink=size_colorbar)
  plt.subplot(3, 2, 5)
  c = plot(sqrt(dot(u_diff,u_diff)), title='Velocity - Difference')
  plt.colorbar(c, orientation='horizontal', ticks=MaxNLocator(nbins=3), format=FormatStrFormatter('%.2f'), pad=pad_space, shrink=size_colorbar)
  
  plt.subplot(3, 2, 2)
  c = plot(p_fom, title='Pressure - FOM')
  plt.colorbar(c, orientation='horizontal', ticks=MaxNLocator(nbins=3), format=FormatStrFormatter('%.2f'), pad=pad_space, shrink=size_colorbar)
  plt.subplot(3, 2, 4)
  c = plot(p_rom, title='Pressure - ROM')
  plt.colorbar(c, orientation='horizontal', ticks=MaxNLocator(nbins=3), format=FormatStrFormatter('%.2f'), pad=pad_space, shrink=size_colorbar)
  plt.subplot(3, 2, 6)
  c = plot(p_diff, title='Pressure - Difference')
  plt.colorbar(c, orientation='horizontal', ticks=MaxNLocator(nbins=3), format=FormatStrFormatter('%.2f'), pad=pad_space, shrink=size_colorbar)

  plt.tight_layout()

  plt.savefig(f"images/solution{n:05}.png")
  plt.clf()
  plt.close(fig)
  
# generate video
if os.path.exists("out.mp4"):
    os.remove("out.mp4")

!ffmpeg -framerate 10 -pattern_type glob -i 'images/*.png' -c:v libx264 -r 30 -pix_fmt yuv420p out.mp4

# play video
from base64 import b64encode
mp4 = open('out.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1000 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)